In [23]:
import keras
#from keras.models import Sequential
#from keras.layers import Conv1D,Conv2D,Dense, Dropout,Flatten,Bidirectional, Activation,BatchNormalization
#from keras.layers import TimeDistributed
from keras.optimizers import SGD

In [24]:
import pickle
x_tr = pickle.load(open("x_data.pk","rb"))
y_tr = pickle.load(open("y_data.pk","rb"))


In [29]:
## data for nanopores
import pickle
pickle_path = "toy_data.pk"

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Conv2D,Dense, Dropout,Flatten,Bidirectional, Activation,BatchNormalization
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.backend import ctc_decode, variable,get_value
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Activation,Input,LSTM, Lambda

n = 100
class_num = 5
batch_size = 10
seq_len = 300

##read nucleotide sequence for each x,y pair and store in arrays
## pad the nucleotide sequences into max_length with 4 (denoting blank)
def read_pickle(pickle_path,example_num = 100 , class_num = 5 , seq_len = 300 ,padding = True):
    all_data = pickle.load(open(pickle_path,"rb"))
    keys = list(all_data.keys())
    #print(all_data[keys[0]]['y_vec'])
    x_tr = []
    y_tr = []
    labels = []
    label_lengths = []
    for key in keys:
        x_tr.append(all_data[key]['x_data'])
        y_tr.append(all_data[key]['y_vec'])
        labels.append(np.array(all_data[key]["nucleotides"])-1)
    x_train = np.array(x_tr[:example_num]).reshape(example_num,seq_len,1)
    y_train = np.array(y_tr[:example_num]).reshape(example_num,seq_len,1)
    y_labels =labels[:example_num]
    label_lengths = list(map(lambda x : len(x),y_labels))
    if padding:
        pad = 4
        max_length = max(list(map(lambda x : len(x),y_labels)))
        for i in range(len(y_labels)):
            leng = len(y_labels[i])
            y_labels[i] = np.pad(y_labels[i],(0,max_length-leng),'constant', constant_values=(4,4))
    #print(y_labels[0])
    y_train_class = keras.utils.to_categorical(y_train,num_classes = class_num)
    return x_train,y_train,y_train_class,y_labels,label_lengths

x_tr,y_tr,y_categorical,y_labels,label_lengths = read_pickle(pickle_path,example_num=n)

In [30]:
assert len(x_tr)== len(y_tr) == len(y_categorical )== len(y_labels) == len(label_lengths), "Dimension not matched"

In [31]:
x_tr.shape

(100, 300, 1)

In [32]:
## conv-layer
def conv1D_layer(inputs,filternum,filtersize,activation='relu'):
    conv = Conv1D(filternum,filtersize,padding='same',input_shape = inputs.shape)
    x = inputs
    x = conv(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x
def chiron_cnn(inputs,filternum1,filtersize1,filtersize2,res_layers = 5,activation = 'relu'):
    x = inputs 
    for i in range(res_layers):
        x = chiron_res_layer(x,filternum1,filtersize1,filtersize2,activation = activation)
    #x = Flatten()(x)
    return x
## two branches of cnn 
def chiron_res_layer(inputs,filternum1,filtersize1,filtersize2,activation = 'relu'):
    x   = inputs 
    b_1 = conv1D_layer(x,filternum1,filtersize1,activation = activation)
    b_2 = conv1D_layer(x,filternum1,filtersize1,activation = activation)
    b_2 = conv1D_layer(b_2,filternum1,filtersize2,activation = activation)
    b_2 = conv1D_layer(b_2,filternum1,filtersize1,activation = activation)
    y = keras.layers.add([b_1,b_2])
    y = Activation(activation)(y)
    return y

def chiron_rnn(inputs,hidden_num =200,rnn_layers = 3,class_num = class_num ):
    x = inputs 
    
    for i in range(rnn_layers):
        x = chiron_bilstm_layer(x,hidden_num = hidden_num)
    #FC = Dense(class_num,activation = 'softmax',input_shape=(hidden_num*2,))
    #x = FC(x)
    return x
def chiron_bilstm_layer(inputs,hidden_num):
    #print(inputs.shape)
    firstbi = Bidirectional(LSTM(hidden_num, return_sequences=True),
                        input_shape=inputs.shape)
    x = inputs 
    x = firstbi(x)
    #print(x)
    x = BatchNormalization()(x)
    return x
inputs = Input(shape=x_tr.shape[1:])
input_length = Input(name='input_length', shape=[1], dtype='int64')
outputs = chiron_cnn(inputs,256,1,3)
#print(outputs.shape)
outputs2 = chiron_rnn(outputs,200)
dense =  TimeDistributed(Dense(class_num))(outputs2)
preds = TimeDistributed(Activation('softmax',name = 'softmax'))(dense)
#print(np.ones(outputs2.shape[1])*int(outputs2.shape[2]))
model2 = Model(inputs= inputs,outputs=preds)
sgd = SGD()
model2.compile(loss = "categorical_crossentropy",optimizer = sgd)
model2.summary()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [83]:
model2.fit(x_train,y_train,batch_size = batch_size,epochs=3)

Epoch 1/3
100/100 [==============================] - 26s 263ms/step - loss: 1.9191
Epoch 2/3
100/100 [==============================] - 15s 154ms/step - loss: 1.6505
Epoch 3/3
100/100 [==============================] - 16s 161ms/step - loss: 1.5112


In [50]:
def ctc_predict(model,inputs,beam_width = 100, top_paths = 1):
    lens = lambda l :list(map (lambda x:len(x),l))
    preds = model.predict(inputs)
    #print(preds)
    if top_paths !=1:
        decoded_preds = ctc_decode(preds,lens(inputs),greedy=False,beam_width=beam_width,top_paths=top_paths)
    else:
        decoded_preds = ctc_decode(preds,lens(inputs),beam_width=beam_width)
    return decoded_preds


In [51]:
decodeds = ctc_predict(model2,x_train[:10])
get_value(decodeds[0][0])

array([[ 2,  1,  2,  3,  1,  1,  2,  1,  0,  1,  0,  2,  1,  1,  3,  2,
         1,  2,  0,  2,  2,  0,  1,  2,  1,  2,  1,  2,  1,  1,  1,  1,
         1,  2,  1, -1, -1, -1, -1, -1],
       [ 3,  1,  2,  1,  2,  1,  3,  2,  0,  2,  2,  0,  3,  0,  0,  3,
         3,  1,  3,  0,  1,  2,  1,  2,  3,  1,  0,  1,  3,  1,  2,  1,
         2,  2,  0, -1, -1, -1, -1, -1],
       [ 2,  3,  2,  2,  3,  3,  1,  3,  1,  3,  1,  3,  1,  2,  3,  2,
         1,  2,  3,  3,  3,  2,  3,  1,  1,  1,  3,  1,  3,  1,  3,  1,
         1,  1,  3,  1,  3,  3,  0,  0],
       [ 0,  3,  2,  3,  1,  3,  2,  1,  0,  0,  0,  2,  2,  1,  2,  1,
         2,  1,  3,  1,  2,  1,  3,  2,  3,  0,  2,  3,  2,  3,  1,  3,
         1,  3,  1, -1, -1, -1, -1, -1],
       [ 0,  3,  2,  2,  0,  2,  3,  2,  1,  2,  1,  2,  0,  1,  2,  3,
         2,  3,  1,  3,  0,  3,  0,  3,  1,  2,  3,  1,  3,  2,  2,  0,
         2, -1, -1, -1, -1, -1, -1, -1],
       [ 2,  1,  2,  1,  3,  0,  2,  0,  2,  2,  3,  3,  1,  2,  3,  1,
   

In [68]:
# Define CTC loss
max_nuc_len = len(y_labels[0])
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
labels = Input(name='the_labels', shape=[max_nuc_len], dtype='int32')
input_length = Input(name='input_length', shape=[1], dtype='int32')
label_length = Input(name='label_length', shape=[1], dtype='int32')
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([preds,labels,input_length,label_length])
flattened_input_x_width = keras.backend.squeeze(input_length, axis=-1)
top_k_decoded, _ = K.ctc_decode(preds, flattened_input_x_width)
decoder = K.function([inputs, flattened_input_x_width], [top_k_decoded[0]])
model3 = Model(inputs= [inputs,labels,input_length,label_length],outputs=loss_out)
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 300, 1)       0                                            
__________________________________________________________________________________________________
conv1d_62 (Conv1D)              (None, 300, 256)     512         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_71 (BatchNo (None, 300, 256)     1024        conv1d_62[0][0]                  
__________________________________________________________________________________________________
activation_77 (Activation)      (None, 300, 256)     0           batch_normalization_71[0][0]     
__________________________________________________________________________________________________
conv1d_63 

In [69]:
#model3.summary()
from keras.optimizers import Adam
model3.compile(loss = {'ctc': lambda y_true, y_pred: y_pred},optimizer = Adam())


In [205]:
flattened_input_x_width = keras.backend.reshape(input_length, (-1,))
flattened_input_x_width = keras.backend.squeeze(input_length, axis=-1)
decoded = ctc_decode(outputs2,flattened_input_x_width)

In [276]:
a = np.array(y_labels)

In [70]:
import random
n = len(x_tr)
#m = max(list(map(lambda x :len(x),y_labels)))
#print(m)
#labels = np.array([[y_labels[j][i] if i<len(y_labels[j]) else 4 for i in range(m)] for j in range(n)])
print(np.array(y_labels).shape)
print(x_tr.shape)
input_lengths = np.array([300 for i in range(n)])
label_lengths = np.array(label_lengths)
outputs = {'ctc': np.zeros(n)}
#inp = np.array([x_train,labels,input_lengths,label_length])
model3.fit([x_tr,np.array(y_labels),np.array(input_lengths),np.array(label_lengths)],outputs,batch_size = batch_size,epochs=10)




(100, 48)
(100, 300, 1)
Epoch 1/10
100/100 [==============================] - 30s 305ms/step - loss: 127.1280
Epoch 2/10
100/100 [==============================] - 19s 185ms/step - loss: 95.3421
Epoch 3/10
100/100 [==============================] - 17s 166ms/step - loss: 80.1018
Epoch 4/10
100/100 [==============================] - 17s 165ms/step - loss: 68.5428
Epoch 5/10
100/100 [==============================] - 16s 164ms/step - loss: 63.8302
Epoch 6/10
100/100 [==============================] - 16s 163ms/step - loss: 62.1846
Epoch 7/10
100/100 [==============================] - 16s 160ms/step - loss: 57.6095
Epoch 8/10
100/100 [==============================] - 16s 157ms/step - loss: 57.1659
Epoch 9/10
100/100 [==============================] - 16s 162ms/step - loss: 54.8204
Epoch 10/10
100/100 [==============================] - 16s 163ms/step - loss: 53.4030


In [75]:
#input_length = x_train[3].shape
#flattened_input_x_width = keras.backend.squeeze(input_length, axis=-1)
for i in range(50):
    inst = i
    pred = decoder([[x_tr[inst]], [300]])
    truth = y_labels[inst]
    print(pred)

[array([[2]])]
[array([[2, 2, 2, 2, 2, 2]])]
[array([[2, 2, 2, 3, 2, 0, 3, 1]])]
[array([[2, 2, 3, 2, 2, 2]])]
[array([[2, 1, 2, 2, 2, 0, 2]])]
[array([[2, 2, 2]])]
[array([[2, 1, 2]])]
[array([[2, 2, 2, 2]])]
[array([[2, 3, 3, 2, 2]])]
[array([[2, 2, 2, 2, 2, 1, 2, 1]])]
[array([[2, 2]])]
[array([[2, 2, 2, 3, 2]])]
[array([[3, 2, 2, 2]])]
[array([[2, 2, 2]])]
[array([[2, 2, 3]])]
[array([[2, 3, 2, 2, 2, 2]])]
[array([[2, 2, 2, 1]])]
[array([[2, 2, 2, 2, 2, 2]])]
[array([[3, 1, 2, 1, 2, 3, 2]])]
[array([[2, 2, 2, 2]])]
[array([[2, 2, 2, 2, 2]])]
[array([[3, 2, 2]])]
[array([[2, 2, 2, 2]])]
[array([[2, 2, 2, 2]])]
[array([[2, 2, 1, 2, 2]])]
[array([[2, 2, 2, 2, 2, 1]])]
[array([[2]])]
[array([[2, 2, 2]])]
[array([[2, 1, 2, 2, 2, 2, 2]])]
[array([[2, 2, 1, 2]])]
[array([[2, 3, 2, 2]])]
[array([[2, 2, 2]])]
[array([[2, 2, 3]])]
[array([[2, 2, 2, 2, 2, 2, 1]])]
[array([[2, 2, 2, 2, 3]])]
[array([[0, 3, 2]])]
[array([[2, 2, 3, 2, 1, 1]])]
[array([[2, 3, 1, 2, 2, 2, 2, 3, 2]])]
[array([[2, 2

In [378]:
model3.predict([[x_tr[0]],np.array(labels[3:4]),np.array(input_lengths[3:4]),np.array(label_lengths[3:4])])

array([[388.05008]], dtype=float32)

In [374]:
def ctc_predict(model,inputs,beam_width = 100, top_paths = 1):
    lens = lambda l :list(map (lambda x:len(x),l))
    preds = model.predict(inputs)
    print(preds)
    if top_paths !=1:
        decoded_preds = ctc_decode(preds,lens(inputs),greedy=False,beam_width=beam_width,top_paths=top_paths)
    else:
        decoded_preds = ctc_decode(preds,lens(inputs),beam_width=beam_width)
    return decoded_preds

In [375]:
b = ctc_predict(model3,[[x_train[3]],np.array(labels[3:4]),np.array(input_lengths[3:4]),np.array(label_lengths[3:4])] ,beam_width = 10,top_paths = 1)




[[412.1857]]


ValueError: Dimension must be 2 but is 3 for 'transpose_48' (op: 'Transpose') with input shapes: [1,1], [3].

In [365]:
get_value(b[0][0])

array([], shape=(1, 0), dtype=int64)

In [176]:
arrs = []
match = 0
for path in b[0]:
    val = get_value(path)
    arrs.append(val[0])
for arr in arrs[]:
    

400

In [171]:
get_value(b[1])

array([[241.06538, 241.06511, 241.06422, 241.06395, 241.06314, 241.06287,
        241.06198, 241.0617 , 241.06091, 241.06064, 241.06055, 241.0599 ,
        241.05988, 241.05975, 241.0596 , 241.05872, 241.05844, 241.05777,
        241.05765, 241.05753]], dtype=float32)